In [2]:
from util.analyze import Analyze
import numpy as np
from pprint import pprint
from pandas import DataFrame


class Bout(object):
    """
    A single walking bout.
    
    Walking-speed-param is calculated over the entire walking bout

    Value of any leg-motion-param is
     averaged over all values
     averaged over all legs in the specified body segment 
    """

    def __init__(self, path):
        """
        
        :param path: path to a single walking bout video that is leg-tracked 
        """
        
        ana = Analyze(path=path)

        self.walking_speed = int(ana.getWalkingSpeed()[0])

        body_segment = ('L2', 'R2', )

        swing_amplitudes = []
        swing_durations = []
        stance_amplitudes = []
        stance_durations = []
        for leg in body_segment:
            swing_amplitudes.extend(ana.getSwingAmplitude(leg))
            swing_durations.extend(ana.getSwingDuration(leg))
            stance_amplitudes.extend(ana.getStanceAmplitude(leg))
            stance_durations.extend(ana.getStanceDuration(leg))

        self.swing_amplitude = np.mean(swing_amplitudes, dtype=np.int)
        self.swing_duration = np.mean(swing_durations, dtype=np.int)
        self.stance_amplitude = np.mean(stance_amplitudes, dtype=np.int)
        self.stance_duration = np.mean(stance_durations, dtype=np.int)

    def df(self):
        return DataFrame.from_records([self.__dict__])

path = '/media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/' \
            'Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/48183 GAL4/top30/01'

bout = Bout(path)
pprint(bout.__dict__)
pprint(bout.df())

{'stance_amplitude': 1450,
 'stance_duration': 96,
 'swing_amplitude': 1571,
 'swing_duration': 33,
 'walking_speed': 14}
   stance_amplitude  stance_duration  swing_amplitude  swing_duration  \
0              1450               96             1571              33   

   walking_speed  
0             14  


In [27]:
from os import listdir
from os.path import isdir
from os.path import join as pjoin


class Trial(object):
    """
    A Trial contains many Bouts of a single genotype
    """
    
    def __init__(self, genotype, root_path):
        print('Analysing: %s' % genotype)
        print('@ %s' % root_path)
        self.genotype = genotype
        paths = [pjoin(root_path, path)
                    for path in listdir(root_path)
                 if isdir(pjoin(root_path, path)) and 'unusable' not in path and 'ignore' not in path]
        
        df = DataFrame()
        for path in paths:
            bout = Bout(path)
            df = df.append(bout.df(), ignore_index=True)
        print('Analysed %i walking bouts' % len(df.index))
        self.df = df

    def save_as_csv(self, path):
        fp = pjoin(path, self.genotype+'.csv')
        self.df.to_csv(fp)
        return fp

trial = Trial(
    genotype='48183-GAL4 X UAS-Rdli',
    root_path='/media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/' \
            'Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/48183 GAL4/top30/'
)
file_path = trial.save_as_csv(
    path='/home/pushkar/Documents/Workspace/freewalk/data'
)
print(file_path)

Analysing: 48183-GAL4 X UAS-Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/48183 GAL4/top30/


Analysed 30 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/48183-GAL4 X UAS-Rdli.csv


In [28]:
output_path = '/home/pushkar/Documents/Workspace/freewalk/data/csv/'
df_table_of_trials = DataFrame.from_csv('/home/pushkar/Documents/Workspace/freewalk/data/TableOfTrials.csv')

for i, row in df_table_of_trials.iterrows():
    trial = Trial(
        genotype=row.genotype,
        root_path=row.root_path
    )
    file_path = trial.save_as_csv(output_path)
    print(file_path)
    print()

Analysing: 48183-GAL4 X UAS-Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/48183 GAL4/top30


Analysed 30 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/48183-GAL4 X UAS-Rdli.csv

Analysing: VGNReg3-GAL4 x UAS-Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/VGN Intron Region 3 GAL4/Consolidated_VGNReg3_Rdli


Analysed 44 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/VGNReg3-GAL4 x UAS-Rdli.csv

Analysing: 48183-GAL4 X UAS-GFP
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/CONTROLS/ 48183 X UAS-GFP/top30


Analysed 30 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/48183-GAL4 X UAS-GFP.csv

Analysing: VGNReg3-GAL4 x UAS-mCD8GFP
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/CONTROLS/VGN-Intron-Region3-GAL4 x UAS_mCD8GFP


Analysed 48 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/VGNReg3-GAL4 x UAS-mCD8GFP.csv

Analysing: OK371-GAL4-TShGal80 X Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/OK371-TShGal80 X Rdl- G RNAi


...

Analysed 28 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/UAS-Rdli x CS.csv

Analysing: UAS-Dicer2;OK371 GAL4 X UAS-Rdl RNAi
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/PanMn/Rdli/Consolidated_UAS-Dicer2;OK371-GAL4 x UAS-Rdli


Analysed 68 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/UAS-Dicer2;OK371 GAL4 X UAS-Rdl RNAi.csv

Analysing: UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-mCD8GFP
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/AdultSpecific/UAS-Dicer2;OK371-GAL4;TubGAL80ts x UAS-mCD8GFP/Consolidated_UAS-Dicer2;OK371-GAl4;TubGAL80ts x UAS-mCD8GFP


Analysed 108 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-mCD8GFP.csv

Analysing: UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-Rdl RNAi
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/AdultSpecific/dicer;OK371;GAl80ts X rdl G-RNAi/consolidated_rdlgi


Analysed 77 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-Rdl RNAi.csv

Analysing: Temperature control; UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-Rdl RNAi
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/AdultSpecific/Temperature_CTRLS/Consolidated_TempCtrls_RDLi


Analysed 9 walking bouts
/home/pushkar/Documents/Workspace/freewalk/data/csv/Temperature control; UAS-Dicer2;OK371 GAL4;TubpGAL80ts X UAS-Rdl RNAi.csv

